# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# ex: from src.create_model_pickle import *


# Import data

In [3]:
# Use our json to dataframe conversion class
from src.pipeline import pipeline_json
pj = pipeline_json('../data/data.json')

In [33]:
df = pj.convert_to_df()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 55 columns):
acct_type              14337 non-null object
approx_payout_date     14337 non-null datetime64[ns]
body_length            14337 non-null int64
channels               14337 non-null int64
country                14256 non-null object
currency               14337 non-null object
delivery_method        14321 non-null float64
description            14337 non-null object
email_domain           14337 non-null object
event_created          14337 non-null datetime64[ns]
event_end              14337 non-null datetime64[ns]
event_published        14238 non-null datetime64[ns]
event_start            14337 non-null datetime64[ns]
fb_published           14337 non-null bool
gts                    14337 non-null float64
has_analytics          14337 non-null bool
has_header             14337 non-null float64
has_logo               14337 non-null bool
listed                 14337 non-null int64
na

date_cols = ['approx_payout_date', 'event_created', 'event_created', 'event_end', \
                     'event_published', 'event_start', 'user_created']

for col in date_cols:
    print df[col][:5] - df['user_created'][:5]

In [ ]:
df['has_header'][:10]
    

a = df['event_published'][:5] - df['user_created'][:5]
a.var

i=12
total_payout = 0
payout_count = len(df['previous_payouts'][i])
payees = set()
for d in df['previous_payouts'][i]:
    total_payout += d['amount']
    payees.add(d['name'])
print total_payout, payout_count, payees

In [ ]:
a = set()

In [ ]:
df['ticket_types'][0][0].keys()

In [ ]:
Counter(df['fraud'])

In [40]:
col = 'ticket_sales_amount'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

ticket_sales_amount
fraud : 461.793078113
not fraud : 2824.76523842


In [41]:
col = 'ticket_sales_count'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

ticket_sales_count
fraud : 4.08120649652
not fraud : 73.8115608709


In [42]:

col = 'ticket_sales_events'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

ticket_sales_events
fraud : 1.0
not fraud : 1.0


In [10]:
df['short_description'][:10]

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: short_description, dtype: bool

In [11]:
df['body_length'][:10]

0     3852
1     3499
2     2601
3    12347
4     2417
5      117
6       28
7      974
8     4388
9      974
Name: body_length, dtype: int64

In [13]:
Counter(df['currency'])

Counter({u'AUD': 779,
         u'CAD': 1322,
         u'EUR': 262,
         u'GBP': 2222,
         u'MXN': 1,
         u'NZD': 55,
         u'USD': 9696})

In [24]:
# table = pivot_table(df, values='D', index=['A', 'B'],
# ...                     columns=['C'], aggfunc=np.sum)
table = pd.pivot_table(df, columns=['currency'], values=['fraud'], aggfunc=np.mean)
print table

currency       AUD      CAD       EUR       GBP  MXN  NZD      USD
fraud     0.015404  0.03177  0.141221  0.154365  1.0  0.0  0.08849


In [23]:
Counter(df['currency'])

Counter({u'AUD': 779,
         u'CAD': 1322,
         u'EUR': 262,
         u'GBP': 2222,
         u'MXN': 1,
         u'NZD': 55,
         u'USD': 9696})

In [39]:
col = 'ticket_sales_events'
print col
print "fraud :", (df[df['fraud'] == True]['event_created'] - df[df['fraud'] == True]['user_created']).mean()
print "not fraud :", (df[df['fraud'] == False]['event_created'] - df[df['fraud'] == False]['user_created']).mean()


ticket_sales_events
fraud : 87 days 03:23:02.605568
not fraud : 402 days 16:24:23.307267


In [30]:
col = 'has_analytics'
print col
print "fraud :", (df[df['fraud'] == True][col]).mean()
print "not fraud :", (df[df['fraud'] == False][col]).mean()


has_analytics
fraud : 0.0030935808198
not fraud : 0.0799601349279


,eu_currency,currency
0,0,USD
1,0,USD
2,0,USD
3,1,EUR
4,0,USD
5,0,USD
6,0,USD
7,0,USD
8,0,USD
9,0,USD
